In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

In [5]:
df = pd.read_csv(r'../data/fake_job_postings.csv')
df.sample(10)

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
16694,16695,Senior Financial Analyst,"US, DC, Washington",NaN,NaN,Rose is a recruiter with TRAK Services in Wash...,Large nonprofit in DC is seeking a talented Sr...,NaN,NaN,0,0,1,NaN,NaN,NaN,NaN,NaN,0
6153,6154,Group HR Manager,"IE, ,",NaN,27500-36000,NaN,"Jordans Centra operates throughout Dublin, Wic...",Human Resource Planning:To implement company p...,NaN,0,0,1,Full-time,Not Applicable,Unspecified,Retail,Human Resources,0
7828,7829,English Teacher Abroad,"US, CA, Santa Rosa",NaN,NaN,We help teachers get safe &amp; secure jobs ab...,"Play with kids, get paid for it Love travel? J...",University degree required. TEFL / TESOL / CEL...,See job description,0,1,1,Contract,NaN,Bachelor's Degree,Education Management,NaN,0
13490,13491,Junior full stack developer,"GR, I, Athens",NaN,NaN,Creators of innovative software products for f...,Duties and responsibilitiesThe nature of work ...,Skills and qualificationsIn order to perform s...,"BDSwiss Labs offers diverse, challenging and e...",0,1,0,Full-time,Entry level,Unspecified,Information Technology and Services,NaN,0
5003,5004,"Controller (Equity, Startup w/ substantial gro...","US, MA, Andover",NaN,130000-145000,NaN,***** Please only apply if you have experience...,The startup road is guaranteed to be long and ...,"keywords: CPA, CA, CFO, Controller, CPG, consu...",0,0,0,Full-time,Director,Bachelor's Degree,Consumer Goods,Finance,0
6199,6200,Embedded Hardware Engineer,"US, MA, Cambridge",NaN,NaN,"Wallflowr is a Cambridge, MA based company wor...",OverviewWe are a self-funded startup based in ...,- 2+ years of relevant work experience in embe...,- Get into a startup early with an experienced...,0,1,1,Full-time,Not Applicable,Bachelor's Degree,Consumer Electronics,Engineering,0
15162,15163,Manufacturing Engineering Manager,"US, PA, Philadelphia",NaN,NaN,We Provide Full Time Permanent Positions for m...,(We have more than 1500+ Job openings in our w...,NaN,NaN,0,0,0,Full-time,NaN,NaN,Electrical/Electronic Manufacturing,NaN,0
1247,1248,Graduates: English Teacher Abroad (Conversatio...,"US, OK, Edmond",NaN,NaN,We help teachers get safe &amp; secure jobs ab...,"Play with kids, get paid for it :-)Vacancies i...",University degree required. TEFL / TESOL / CEL...,See job description,0,1,1,Contract,Entry level,Bachelor's Degree,Education Management,Education,0
1889,1890,Controls Engineer- II,"US, MI, Milford Township",Computer/IT Services,NaN,NaN,This position involves manipulating test vehic...,Masters or PhD in Electrical/Computer Engineer...,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,0
10689,10690,Copywriter,"US, NY,",NaN,NaN,Namely is the leading end-to-end HR and payrol...,Namely is an agile startup dedicated to buildi...,Who are you:Technology and Internet savvy with...,"Ping pong/foosball, dogs, beer, Brooklyn/Manha...",0,1,0,Full-time,Associate,NaN,Information Technology and Services,NaN,0
